In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

In [2]:
role = get_execution_role()
sess = sagemaker.Session()

변수 설정
manifest 파일이 위치한 곳을 지정하세요.

In [3]:
bucket = sess.default_bucket() 
print(bucket)

prefix = 'motocycle_object_detection'

train_manifest = 'output_datasets/train.manifest'
validation_manifest = 'output_datasets/validation.manifest'

label_key_name = 'kr-reg-object-dection'

s3_train_annotation = 's3://{}/{}/{}'.format(bucket, prefix, train_manifest)
s3_validation_annotation = 's3://{}/{}/{}'.format(bucket, prefix, validation_manifest)
s3_output_location = 's3://{}/{}/train_output'.format(bucket, prefix)

In [7]:
training_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version="latest")

od_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p2.xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode= 'Pipe',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [8]:
od_model.set_hyperparameters(base_network='resnet-50',
                             use_pretrained_model=1,
                             num_classes=2,
                             mini_batch_size=10,
                             epochs=800,
                             learning_rate=0.01,
                             lr_scheduler_step='3,6',
                             lr_scheduler_factor=0.1,
                             optimizer='sgd',
                             momentum=0.9,
                             weight_decay=0.0005,
                             overlap_threshold=0.5,
                             nms_threshold=0.45,
                             image_shape=300,
                             label_width=350,
                             num_training_samples=21)

In [9]:
train_data = sagemaker.session.s3_input(s3_train_annotation, distribution='FullyReplicated', content_type='application/x-recordio', record_wrapping = 'RecordIO',
s3_data_type='AugmentedManifestFile', attribute_names=['source-ref', label_key_name])

validation_data = sagemaker.session.s3_input(s3_validation_annotation, distribution='FullyReplicated', content_type='application/x-recordio', record_wrapping = 'RecordIO',
s3_data_type='AugmentedManifestFile', attribute_names=['source-ref', label_key_name])

data_channels = {'train': train_data, 'validation': validation_data}

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [ ]:
od_model.fit(inputs=data_channels, logs=True)

2020-09-24 01:35:06 Starting - Starting the training job...
2020-09-24 01:35:09 Starting - Launching requested ML instances......
2020-09-24 01:36:33 Starting - Preparing the instances for training.........
2020-09-24 01:37:56 Downloading - Downloading input data
2020-09-24 01:37:56 Training - Downloading the training image........Docker entrypoint called with argument(s): train
[09/24/2020 01:39:16 INFO 140556653164352] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'label_width': u'350', u'early_stopping_min_epochs': u'10', u'epochs': u'30', u'overlap_threshold': u'0.5', u'lr_scheduler_factor': u'0.1', u'_num_kv_servers': u'auto', u'weight_decay': u'0.0005', u'mini_batch_size': u'32', u'use_pretrained_model': u'0', u'freeze_layer_pattern': u'', u'lr_scheduler_step': u'', u'early_stopping': u'False', u'early_stopping_patience': u'5', u'momentum': u'0.9', u'num_training_samples': u'', u'optimizer': u'sgd', u'_tuning_objective_

In [ ]:
object_detector = od_model.deploy(initial_instance_count = 1,
                                 instance_type = 'ml.p2.xlarge')